In [4]:
from src.Base.Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from src.Utils.ICM_preprocessing import *
from src.Utils.load_ICM import load_ICM
from src.Utils.load_URM import load_URM
import sklearn

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample

URMs_train = []
URMs_validation = []

for k in range(5):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)

evaluator_validation = K_Fold_Evaluator_MAP(URMs_validation, cutoff_list=[10], verbose=False)

ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))

from src.Hybrid.GeneralizedSimilarityMergedHybridRecommender import GeneralizedSimilarityMergedHybridRecommender
from src.Hybrid.MergedHybridRecommender import MergedHybridRecommender
from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from src.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

In [5]:
p3alpha_recommenders = []
for index in range(len(URMs_train)):

    p3alpha_recommenders.append(
        P3alphaRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    p3alpha_recommenders[index].fit(
        topK=int(212.8832860130684),
        alpha=0.4729294763382114,
        implicit=True
    )

result = evaluator_validation.evaluateRecommender(p3alpha_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

MAP = 0.05938172002584221


In [6]:
rp3betaCombined_recommenders = []
for index in range(len(URMs_train)):
    rp3betaCombined_recommenders.append(
        RP3betaCBFRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICMs_combined[index],
            verbose=False
        )
    )

    rp3betaCombined_recommenders[index].fit(
        topK=int(525.3588205773788),
        alpha=0.42658191175355076,
        beta=0.2284685880641364,
        implicit=False
    )

result = evaluator_validation.evaluateRecommender(rp3betaCombined_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

MAP = 0.07254691289113213


In [7]:
rp3betaCBF_recommenders = []
for index in range(len(URMs_train)):

    rp3betaCBF_recommenders.append(
        RP3betaCBFRecommender(
            URM_train=URMs_train[index],
            ICM_train=ICM_all,
            verbose=False
        )
    )

    rp3betaCBF_recommenders[index].fit(
        topK=int(58.16144182493173),
        alpha=0.26520214286626453,
        beta=0.36456352256640157,
        implicit=False
    )

result = evaluator_validation.evaluateRecommender(rp3betaCBF_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

MAP = 0.038662950572284374


In [8]:
itemKNN_recommenders = []
for index in range(len(URMs_train)):
    itemKNN_recommenders.append(
        ItemKNNCFRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    itemKNN_recommenders[index].fit(
        topK=100,
        shrink=50
    )

result = evaluator_validation.evaluateRecommender(itemKNN_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

Similarity column 25975 ( 100 % ), 7373.20 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 7438.75 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 7298.56 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 7389.99 column/sec, elapsed time 0.06 min
Similarity column 25975 ( 100 % ), 7151.78 column/sec, elapsed time 0.06 min
MAP = 0.0496071118989157


In [9]:
pureSVD_recommenders = []
for index in range(len(URMs_train)):
    pureSVD_recommenders.append(
        PureSVDItemRecommender(
            URM_train=URMs_train[index],
            verbose=False
        )
    )

    pureSVD_recommenders[index].fit(
        num_factors=772,
        topK= 599
    )


result = evaluator_validation.evaluateRecommender(pureSVD_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

MAP = 0.03421253920358079


In [41]:
slimBPRCombined_recommenders = []
for index in range(len(URMs_train)):
    slimBPRCombined_recommenders.append(
        SLIM_BPR_Cython(
            URM_train=ICMs_combined[index].T,
            verbose=False
        )
    )

    slimBPRCombined_recommenders[index].fit(
        epochs=39.54,
        positive_threshold_BPR=None,
        train_with_sparse_weights=True,
        symmetric=False,
        random_seed=None,
        batch_size=393.93324941229486,
        lambda_i=0.004419,
        lambda_j=0.001592,
        learning_rate=1e-4,
        topK=int(891.9),
        sgd_mode='adagrad',
        # gamma=,
        # beta_1=,
        # beta_2=,
    )



    slimBPRCombined_recommenders[index].URM_train = URMs_train[index]
    slimBPRCombined_recommenders[index].W_sparse = sklearn.preprocessing.normalize(
        slimBPRCombined_recommenders[index].W_sparse,
        norm='l2',
        axis=1,
        copy=False,
        return_norm=False
    )

result = evaluator_validation.evaluateRecommender(slimBPRCombined_recommenders)
print('MAP = {}'.format(sum(result) / len(result)))

SLIM_BPR_Recommender: Epoch 1 of 39.54. Elapsed time 0.68 sec
SLIM_BPR_Recommender: Epoch 2 of 39.54. Elapsed time 2.08 sec
SLIM_BPR_Recommender: Epoch 3 of 39.54. Elapsed time 3.96 sec
SLIM_BPR_Recommender: Epoch 4 of 39.54. Elapsed time 6.49 sec
SLIM_BPR_Recommender: Epoch 5 of 39.54. Elapsed time 10.04 sec
SLIM_BPR_Recommender: Epoch 6 of 39.54. Elapsed time 14.54 sec
SLIM_BPR_Recommender: Epoch 7 of 39.54. Elapsed time 19.86 sec
SLIM_BPR_Recommender: Epoch 8 of 39.54. Elapsed time 25.95 sec
SLIM_BPR_Recommender: Epoch 9 of 39.54. Elapsed time 32.82 sec
SLIM_BPR_Recommender: Epoch 10 of 39.54. Elapsed time 40.86 sec
SLIM_BPR_Recommender: Epoch 11 of 39.54. Elapsed time 49.95 sec
SLIM_BPR_Recommender: Epoch 12 of 39.54. Elapsed time 1.00 min
SLIM_BPR_Recommender: Epoch 13 of 39.54. Elapsed time 1.19 min
SLIM_BPR_Recommender: Epoch 14 of 39.54. Elapsed time 1.41 min
SLIM_BPR_Recommender: Epoch 15 of 39.54. Elapsed time 1.65 min
SLIM_BPR_Recommender: Epoch 16 of 39.54. Elapsed time 1.9

In [50]:
recommenders = []
for index in range(len(URMs_train)):
    recommenders.append(
        GeneralizedSimilarityMergedHybridRecommender(
        URM_train=URMs_train[index],
        similarityRecommenders=[
            p3alpha_recommenders[index],
            rp3betaCombined_recommenders[index],
            rp3betaCBF_recommenders[index],
            itemKNN_recommenders[index],
            pureSVD_recommenders[index],
            slimBPRCombined_recommenders[index]
        ],
        verbose=False
    )
    )

    recommenders[index].fit(
        topKs=[
            738,
            796,
            896,
            1495,
            0
        ],
        alphas=[
            0.2,
            0.95,
            0.8,
            0.95,
            0.99
        ]
    )

result = evaluator_validation.evaluateRecommender(recommenders)
print(result)
print('MAP = {}'.format(sum(result) / len(result)))






[0.07616656524308091, 0.07708244936201979, 0.07500440810632926, 0.07342513205544475, 0.07648747527876883]
MAP = 0.0756332060091287
